# BenzDB tools

## Source code

### Module imports

In [1]:
import ipywidgets as w
import requests
import json
import base64 
import io
from PIL import Image
import IPython as ip
import jupyter_jsmol as Jsmol

### Definition of criterion classes

#### Definition of the main criterion class

In [2]:
class Criterion:
    """ This class allows for representing criterions """
    
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        self.__key = key
        self.__description  = description     
    
    
    def display (self):
        """ displays the widget corresponding to the criterion """
        pass
    
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        pass
    
    
    def get_description (self) -> str:
        """ returns the description """
        return self.__description
    
    def get_key (self) -> str:
        """ returns the key related to the criterion """
        return self.__key

#### Definition of criterion child classes

In [3]:
class Int_Criterion (Criterion):
    """ This class allows for representing criterions based on int value """
 
    def __init__ (self, key: str, description: str, initial_value: int):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(value=1, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [4]:
class Float_Criterion (Criterion):
    """ This class allows for representing criterions based on float value """
 
    def __init__ (self, key: str, description: str, initial_value: float):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [5]:
class String_Criterion (Criterion):
    """ This class allows for representing criterions based on string value """
 
    def __init__ (self, key: str, description: str, initial_value: int):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(value=1, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>"], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [6]:
class Query_Criterion (Criterion):
    """ This class allows for representing the desired query """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Select(description=self.get_description(), options=["benzenoids","ir","ims2d1a","nics","clar_covers","properties","irregularities"], value='benzenoids', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        return str(self.__element.value)

### Definition of the form class

In [7]:
class Form:
    
    def __init__ (self):
        """ initialise the form """
        self.__query = None         # the query
        self.__demand_type = None   # the type of demand
        self.__json_string = None   # the query as a JSON string
        self.__data = None          # the data related to the query (if the query succeeds)
        
        self.create_form()
        
        
    def create_form(self) -> None:
        """ creates the form allowing for choosing the values of desired criteria """
        self.__criteria = []

        # criteria about basic information
        self.__criteria.append (Int_Criterion(key="nbHexagons", description="# hexagons", initial_value=1))
        self.__criteria.append (Int_Criterion(key="nbCarbons", description="# carbons", initial_value=1))
        self.__criteria.append (Int_Criterion(key="nbHydrogens", description="# hydrogens", initial_value=1))

        # query
        self.__criteria.append (Query_Criterion(key="query", description="query"))    

        # validation buttons
        count_btn = w.Button (description="Count")
        count_btn.on_click (self.perform_query)

        getdata_btn = w.Button (description="Get Data")
        getdata_btn.on_click (self.perform_query)

        getquery_btn = w.Button (description="Get JSON Query")
        getquery_btn.on_click (self.perform_query)
        
        box = w.HBox([count_btn, getdata_btn,getquery_btn])
        display(box)
    
    
    def perform_query (self, btn) -> None:
        """ performs the query and sets the corresponding attributes """ 
        # we identify the type of demand
        if btn.description == "Count":
            self.__demand_type = "count"
        elif btn.description == "Get Data":
            self.__demand_type = "data"
        elif btn.description == "Get JSON Query":
            self.__demand_type = "json"
        else:
            self.__demand_type = "unknown"

        # we build the JSON string and the corres
        self.__json_string = "{\n"
        for c in self.__criteria:
            if isinstance(c,Query_Criterion):
                self.__query = c.get_criterion()
            else:
                s = c.get_criterion()
                if len(s) > 0:
                    if len(self.__json_string) > 3:
                        self.__json_string += ",\n"
                    self.__json_string += "\t" + s
        self.__json_string += "\n}"

        if self.__demand_type == "data":
            response = requests.post("https://benzenoids.lis-lab.fr/find_"+self.__query, json= json.loads(self.__json_string))
        elif self.__demand_type == "count":
            response = requests.post("https://benzenoids.lis-lab.fr/count_"+self.__query, json= json.loads(self.__json_string))
        
        if self.__demand_type in ["data","count"]:
            if response.status_code == 200:
                self.__data = response.json()

        
    def get_data (self):
        """ returns the data related to the query """
        return self.__data
    
    
    def get_query (self):
        """ returns the desired query """
        return self.__query

    
    def get_json_string (self):
        """ returns the JSON string related to the desired query """
        return self.__json_string
    

    def get_demand_type (self):
        """ returns the type of the current demand """
        return self.__demand_type

### Definition of the display classes

In [8]:
class Display:
    """ This class allows for displaying benzenoid information """
    
    def __init__ (self, info: dict):
        """ initializes the display tool """
        self.__information = info
        
    
    def display (self) -> None:
        """ displays the information """
        print ("Benzenoid #",self.get_information("idBenzenoid"), sep="")
        print ("InChI:",self.get_information("inchi"))
        print ("Label:",self.get_information("label"))
        print ("#hexagons:",self.get_information("nbHexagons"))
        print ("#carbons:",self.get_information("nbCarbons"))
        print ("#hydrogens:",self.get_information("nbHydrogens"))
        print ("Weight:",self.get_information("weight"))
        print ("Irregularity:",self.get_information("irregularity"))
        print ("Graph file:",self.get_information("graphFile"))
        print ("Geometry:",self.get_information("geometry"))
        self.display_molecule()
    
    
    def display_molecule (self) -> None:
        """ displays the molecule thanks to Jsmol """
        view = Jsmol.JsmolView.from_str(str(self.get_information("nbCarbons")+self.get_information("nbHydrogens"))+"\nComment\n"+self.get_information("geometry"))
        ip.display.display(view)
        
    
    def display_image (self, str64: str) -> None:
        """ displays the base-64 image defined by str64 """
        img = Image.open(io.BytesIO(base64.b64decode(str64)))
        ip.display.display(img)
        
    
    def get_information (self, key) -> dict:
        """ returns the information """
        return self.__information[key]

In [9]:
class Display_IR (Display):
    """ This class allows for displaying benzenoid information from IR query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        print("Final energy:",self.get_information("finalEnergy"))
        print("Frequencies:",self.get_information("frequencies")) 
        print("Intensities:",self.get_information("intensities"))
        print("AMES Format:",self.get_information("amesFormat"))

In [10]:
class Display_IMS2D1A (Display):
    """ This class allows for displaying benzenoid information from IMS2D1A query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        print ("VectorX:",self.get_information("vectorX"))
        print ("VectorY:",self.get_information("vectorY"))
        print ("#PointsX:",self.get_information("nbPointsX"))
        print ("#PointsY:",self.get_information("nbPointsY"))
        print ("Origin:",self.get_information("origin"))
        print ("Points:",self.get_information("points"))
        print ("Type:",self.get_information("type"))
        self.display_image(self.get_information("picture"))

In [11]:
class Display_NICS (Display):
    """ This class allows for displaying benzenoid information from NICS query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        print ("NICS values:", self.get_information("nics"))

In [12]:
class Display_Clar_Covers (Display):
    """ This class allows for displaying benzenoid information from Clar Cover query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        self.display_image(self.get_information("clarCover"))

In [13]:
class Display_Properties (Display):
    """ This class allows for displaying benzenoid information from properties query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        yesno = ["no","yes"]
        print ("Catacondensed:",yesno[self.get_information("catacondensed")])
        print ("Coronoid:",yesno[self.get_information("coronoid")])
        print ("Coronenoid:",yesno[self.get_information("coronenoid")])
        print ("Symmetry:",self.get_information("symmetry"))
        print ("#Kekulé structures:",self.get_information("kekule"))
        print ("HOMO:",self.get_information("homo"))
        print ("LUMO:",self.get_information("lumo"))
        print ("Dipole moment:",self.get_information("moment"))

In [14]:
class Display_Irregularities (Display):
    """ This class allows for displaying benzenoid information from properties query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        'solo', 'duo', 'trio', 'quartet'
        print ("# solo:",self.get_information("solo"))
        print ("# duo:",self.get_information("duo"))
        print ("# trio:",self.get_information("trio"))
        print ("# quartet:",self.get_information("quartet"))
        

## Filling the form

In [15]:
form = Form()


## Processing the result molecule per molecule

In [16]:
demand_type = form.get_demand_type()

if demand_type == "data":
    data = form.get_data()
    query = form.get_query()
    
    for molecule in data:
        if query == "benzenoids":
            Display(molecule).display()
        elif query == "ir":
            Display_IR(molecule).display()
        elif query == "ims2d1a":
            Display_IMS2D1A(molecule).display()
        elif query == "nics":
            Display_NICS(molecule).display()
        elif query == "clar_covers":
            Display_Clar_Covers(molecule).display()
        elif query == "properties":
            Display_Properties(molecule).display()
        elif query == "irregularities":
            Display_Irregularities(molecule).display()
        print()
        
elif demand_type == "count":
    print ("Number of molecules:",form.get_data())
    
elif demand_type == "json":
    print ("JSON query:")
    print (form.get_json_string())

Benzenoid #475
InChI: 1S/C18H12/c1-2-8-14-13(7-1)15-9-3-4-11-17(15)18-12-6-5-10-16(14)18/h1-12H
Label: 1-5-6-12
#hexagons: 4
#carbons: 18
#hydrogens: 12
Weight: 228.2886
Irregularity: 1.0
Graph file: p DIMACS 18 21 4
e 0_0 1_1
e 0_0 -1_1
e 1_1 1_2
e 1_2 0_3
e 0_3 -1_2
e 0_3 0_4
e -1_2 -1_1
e -1_2 -2_3
e -3_2 -2_3
e -3_2 -4_3
e -2_3 -2_4
e -2_4 -3_5
e -2_4 -1_5
e -3_5 -4_4
e -4_4 -4_3
e 0_4 -1_5
e 0_4 1_5
e -1_5 -1_6
e 1_5 1_6
e 1_6 0_7
e 0_7 -1_6
h 0_0 1_1 1_2 0_3 -1_2 -1_1 
h -3_2 -2_3 -2_4 -3_5 -4_4 -4_3 
h -1_2 0_3 0_4 -1_5 -2_4 -2_3 
h 0_4 1_5 1_6 0_7 -1_6 -1_5 

Geometry: C 0.70214 3.70926 0.00000
C 1.38834 2.50364 0.00000
C -0.70214 3.70926 -0.00000
C 0.71283 1.25934 0.00000
C -0.71283 1.25934 -0.00000
C 1.44704 -0.01235 -0.00000
C -1.38834 2.50364 -0.00000
C -1.44704 -0.01235 -0.00000
C 0.73421 -1.24700 0.00000
C 2.86239 -0.04948 -0.00000
C -0.73421 -1.24700 0.00000
C 1.47405 -2.45416 0.00000
C -1.47405 -2.45416 0.00000
C -2.86239 -0.04948 -0.00000
C 3.56338 -1.24656 -0.00000
C 

JsmolView(layout=Layout(align_self='stretch', height='400px'))


Benzenoid #5036
InChI: 1S/C18H12/c1-2-7-15-12-18-16(11-14(15)6-1)10-9-13-5-3-4-8-17(13)18/h1-12H
Label: 0-6-12-17
#hexagons: 4
#carbons: 18
#hydrogens: 12
Weight: 228.2886
Irregularity: 0.6667
Graph file: p DIMACS 18 21 4
e 0_0 1_1
e 0_0 -1_1
e 1_1 1_2
e 1_2 0_3
e 1_2 2_3
e 0_3 -1_2
e 0_3 0_4
e -1_2 -1_1
e 2_3 2_4
e 2_4 1_5
e 2_4 3_5
e 1_5 0_4
e 1_5 1_6
e 3_5 3_6
e 3_5 4_4
e 3_6 2_7
e 3_6 4_7
e 2_7 1_6
e 4_4 5_5
e 5_5 5_6
e 5_6 4_7
h 0_0 1_1 1_2 0_3 -1_2 -1_1 
h 1_2 2_3 2_4 1_5 0_4 0_3 
h 2_4 3_5 3_6 2_7 1_6 1_5 
h 4_4 5_5 5_6 4_7 3_6 3_5 

Geometry: C -2.48946 0.51591 0.00000
C -1.29552 -0.29408 -0.00000
C -2.42296 1.87377 0.00000
C -0.00000 0.34988 0.00000
C -1.37816 -1.68872 -0.00000
C 0.06183 1.81614 0.00000
C 1.13760 -0.46030 -0.00000
C -1.15539 2.56453 0.00000
C 1.28143 2.53305 0.00000
C -1.10739 3.97841 0.00000
C -0.23100 -2.50469 -0.00000
C 1.06307 -1.86918 -0.00000
C -0.29871 -3.93223 -0.00000
C 2.23150 -2.69223 -0.00000
C 0.84895 -4.69248 -0.00000
C 2.12885 -4.06518 -0.00000

JsmolView(layout=Layout(align_self='stretch', height='400px'))


Benzenoid #5239
InChI: 1S/C18H12/c1-3-7-15-13(5-1)9-11-18-16-8-4-2-6-14(16)10-12-17(15)18/h1-12H
Label: 0-6-11-17
#hexagons: 4
#carbons: 18
#hydrogens: 12
Weight: 228.2886
Irregularity: 0.6667
Graph file: p DIMACS 18 21 4
e 0_0 1_1
e 0_0 -1_1
e 1_1 1_2
e 1_2 0_3
e 1_2 2_3
e 0_3 -1_2
e 0_3 0_4
e -1_2 -1_1
e 2_3 2_4
e 2_3 3_2
e 2_4 1_5
e 2_4 3_5
e 1_5 0_4
e 3_2 4_3
e 4_3 4_4
e 4_4 3_5
e 4_4 5_5
e 3_5 3_6
e 5_5 5_6
e 5_6 4_7
e 4_7 3_6
h 0_0 1_1 1_2 0_3 -1_2 -1_1 
h 1_2 2_3 2_4 1_5 0_4 0_3 
h 3_2 4_3 4_4 3_5 2_4 2_3 
h 4_4 5_5 5_6 4_7 3_6 3_5 

Geometry: C 0.64801 3.92849 0.00000
C 0.63283 2.50918 0.00000
C -0.52916 4.65296 0.00000
C -0.61653 1.80977 0.00000
C 1.84936 1.75893 0.00000
C -1.80686 2.58662 0.00000
C -0.61653 0.35446 0.00000
C -1.76648 3.97097 0.00000
C 1.84058 0.39156 0.00000
C 0.61653 -0.35446 0.00000
C 0.61653 -1.80977 0.00000
C -1.84058 -0.39156 0.00000
C -0.63283 -2.50918 0.00000
C 1.80686 -2.58662 0.00000
C -1.84936 -1.75893 0.00000
C -0.64801 -3.92849 0.00000
C 1.76648 

...

JsmolView(layout=Layout(align_self='stretch', height='400px'))


Benzenoid #8010
InChI: 1S/C16H10/c1-3-11-7-9-13-5-2-6-14-10-8-12(4-1)15(11)16(13)14/h1-10H
Label: 0-5-6-11
#hexagons: 4
#carbons: 16
#hydrogens: 10
Weight: 202.25124
Irregularity: 0.6
Graph file: p DIMACS 16 19 4
e 0_0 1_1
e 0_0 -1_1
e 1_1 1_2
e 1_1 2_0
e 1_2 0_3
e 1_2 2_3
e 0_3 -1_2
e 0_3 0_4
e -1_2 -1_1
e 2_0 3_1
e 3_1 3_2
e 3_2 2_3
e 3_2 4_3
e 2_3 2_4
e 2_4 1_5
e 2_4 3_5
e 1_5 0_4
e 4_3 4_4
e 4_4 3_5
h 0_0 1_1 1_2 0_3 -1_2 -1_1 
h 2_0 3_1 3_2 2_3 1_2 1_1 
h 1_2 2_3 2_4 1_5 0_4 0_3 
h 3_2 4_3 4_4 3_5 2_4 2_3 

Geometry: C 0.00000 2.46945 -0.68262
C 0.00000 1.23929 -1.43213
C -0.00000 2.46945 0.68262
C 0.00000 0.00000 -0.71477
C 0.00000 1.21370 -2.83949
C -0.00000 0.00000 0.71477
C 0.00000 -1.23929 -1.43213
C -0.00000 1.23929 1.43213
C 0.00000 -1.23929 1.43213
C -0.00000 1.21370 2.83949
C 0.00000 -0.00000 -3.53182
C 0.00000 -1.21370 -2.83949
C 0.00000 -2.46945 -0.68262
C 0.00000 -1.21370 2.83949
C 0.00000 -2.46945 0.68262
C -0.00000 0.00000 3.53182
H 0.00000 3.40595 -1.23293
H -0.000

JsmolView(layout=Layout(align_self='stretch', height='400px'))


Benzenoid #8267
InChI: 1S/C18H12/c1-2-6-14-10-18-12-16-8-4-3-7-15(16)11-17(18)9-13(14)5-1/h1-12H
Label: 0-6-12-18
#hexagons: 4
#carbons: 18
#hydrogens: 12
Weight: 228.2886
Irregularity: 0.6667
Graph file: p DIMACS 18 21 4
e 0_0 1_1
e 0_0 -1_1
e 1_1 1_2
e 1_2 0_3
e 1_2 2_3
e 0_3 -1_2
e 0_3 0_4
e -1_2 -1_1
e 2_3 2_4
e 2_4 1_5
e 2_4 3_5
e 1_5 0_4
e 1_5 1_6
e 3_5 3_6
e 3_6 2_7
e 3_6 4_7
e 2_7 1_6
e 2_7 2_8
e 4_7 4_8
e 4_8 3_9
e 3_9 2_8
h 0_0 1_1 1_2 0_3 -1_2 -1_1 
h 1_2 2_3 2_4 1_5 0_4 0_3 
h 2_4 3_5 3_6 2_7 1_6 1_5 
h 3_6 4_7 4_8 3_9 2_8 2_7 

Geometry: C -0.00000 4.90023 -0.71651
C -0.00000 3.71870 -1.41255
C -0.00000 4.90023 0.71651
C -0.00000 2.45587 -0.72762
C -0.00000 2.45587 0.72762
C -0.00000 1.23762 -1.40992
C -0.00000 3.71870 1.41255
C -0.00000 1.23762 1.40992
C -0.00000 -0.00000 -0.72808
C -0.00000 0.00000 0.72808
C 0.00000 -1.23762 -1.40992
C 0.00000 -1.23762 1.40992
C 0.00000 -2.45587 -0.72762
C 0.00000 -2.45587 0.72762
C 0.00000 -3.71870 -1.41255
C 0.00000 -3.71870 1.41255
C

JsmolView(layout=Layout(align_self='stretch', height='400px'))